# Marketing Campaign Analysis
This project explores marketing campaign performance through a structured sequence of business driven questions and answers, examining how campaigns generate user engagement, convert interest into purchases, operate efficiently, and differ in performance across geographic markets

## Dataset Source
The dataset used in this project was obtained from https://www.kaggle.com/datasets/alperenmyung/social-media-advertisement-performance.


## Load Data

In [1]:
import pandas as pd
import duckdb

# Load files
ad_events = pd.read_csv("ad_events.csv", parse_dates=["timestamp"])
ads = pd.read_csv("ads.csv")
campaigns = pd.read_csv("campaigns.csv", parse_dates=["start_date", "end_date"])
users = pd.read_csv("users.csv")

# Register tables
con = duckdb.connect()
con.register("ad_events", ad_events)
con.register("ads", ads)
con.register("campaigns", campaigns)
con.register("users", users)

def q(sql: str):
    return con.execute(sql).df()


## Quick Data Overview

In [2]:
for name in ["ad_events", "ads", "campaigns", "users"]:
    print(f"{name}: {con.execute(f'SELECT COUNT(*) FROM {name}').fetchone()[0]:,} rows")

for name in ["ad_events", "ads", "campaigns", "users"]:
    display(q(f"DESCRIBE {name}")[["column_name", "column_type"]])

for name in ["ad_events", "ads", "campaigns", "users"]:
    display(q(f"SELECT * FROM {name} LIMIT 5"))


ad_events: 400,000 rows
ads: 200 rows
campaigns: 50 rows
users: 10,000 rows


,column_name,column_type
0,event_id,BIGINT
1,ad_id,BIGINT
2,user_id,VARCHAR
3,timestamp,TIMESTAMP_NS
4,day_of_week,VARCHAR
5,time_of_day,VARCHAR
6,event_type,VARCHAR


,column_name,column_type
0,ad_id,BIGINT
1,campaign_id,BIGINT
2,ad_platform,VARCHAR
3,ad_type,VARCHAR
4,target_gender,VARCHAR
5,target_age_group,VARCHAR
6,target_interests,VARCHAR


,column_name,column_type
0,campaign_id,BIGINT
1,name,VARCHAR
2,start_date,TIMESTAMP_NS
3,end_date,TIMESTAMP_NS
4,duration_days,BIGINT
5,total_budget,DOUBLE


,column_name,column_type
0,user_id,VARCHAR
1,user_gender,VARCHAR
2,user_age,BIGINT
3,age_group,VARCHAR
4,country,VARCHAR
5,location,VARCHAR
6,interests,VARCHAR


,event_id,ad_id,user_id,timestamp,day_of_week,time_of_day,event_type
0,1,197,2359b,2025-07-26 00:19:56,Saturday,Night,Like
1,2,51,f9c67,2025-06-15 08:28:07,Sunday,Morning,Share
2,3,46,5b868,2025-06-27 00:40:02,Friday,Night,Impression
3,4,166,3d440,2025-06-05 19:20:45,Thursday,Evening,Impression
4,5,52,68f1a,2025-07-22 08:30:29,Tuesday,Morning,Impression


,ad_id,campaign_id,ad_platform,ad_type,target_gender,target_age_group,target_interests
0,1,28,Facebook,Video,Female,35-44,"art, technology"
1,2,33,Facebook,Stories,All,25-34,"travel, photography"
2,3,20,Instagram,Carousel,All,25-34,technology
3,4,28,Facebook,Stories,Female,25-34,news
4,5,24,Instagram,Image,Female,25-34,news


,campaign_id,name,start_date,end_date,duration_days,total_budget
0,1,Campaign_1_Launch,2025-05-25,2025-07-23,59,24021.32
1,2,Campaign_2_Launch,2025-04-16,2025-07-07,82,79342.41
2,3,Campaign_3_Winter,2025-05-04,2025-06-29,56,14343.25
3,4,Campaign_4_Summer,2025-06-04,2025-08-08,65,45326.60
4,5,Campaign_5_Launch,2025-07-11,2025-08-28,48,68376.69


,user_id,user_gender,user_age,age_group,country,location,interests
0,a2474,Female,24,18-24,United Kingdom,New Mariomouth,"fitness, health"
1,141e5,Male,21,18-24,Germany,Danielsfort,"food, fitness, lifestyle"
2,34db0,Male,27,25-34,Australia,Vincentchester,"fashion, news"
3,20d08,Female,28,25-34,India,Lisaport,"health, news, finance"
4,9e830,Male,28,25-34,United States,Brownmouth,"health, photography, lifestyle"


## Data Integrity Check

### Duplicated rows

In [3]:
for table in ["ad_events", "ads", "campaigns", "users"]:
    total = con.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    distinct = con.execute(f"SELECT COUNT(*) FROM (SELECT DISTINCT * FROM {table})").fetchone()[0]
    print(f"{table}: {total - distinct:,} duplicated rows")


ad_events: 0 duplicated rows
ads: 0 duplicated rows
campaigns: 0 duplicated rows
users: 0 duplicated rows


### Missing values

In [4]:
for table in ["ad_events", "ads", "campaigns", "users"]:
    display(q(f"SELECT * FROM {table} LIMIT 100000").isna().sum())


event_id       0
ad_id          0
user_id        0
timestamp      0
day_of_week    0
time_of_day    0
event_type     0
dtype: int64

ad_id               0
campaign_id         0
ad_platform         0
ad_type             0
target_gender       0
target_age_group    0
target_interests    0
dtype: int64

campaign_id      0
name             0
start_date       0
end_date         0
duration_days    0
total_budget     0
dtype: int64

user_id        0
user_gender    0
user_age       0
age_group      0
country        0
location       0
interests      0
dtype: int64

## Baseline Overview

### Question
What time coverage and scale does the dataset include?

In [5]:
sql = r'''SELECT
  MIN(ad_events.timestamp) AS start_timestamp,
  MAX(ad_events.timestamp) AS end_timestamp,
  COUNT(DISTINCT ad_events.user_id) AS users,
  COUNT(DISTINCT ad_events.ad_id) AS ads,
  COUNT(DISTINCT ads.campaign_id) AS campaigns,
  COUNT(*) AS total_events
FROM ad_events
JOIN ads ON ad_events.ad_id = ads.ad_id;'''
display(q(sql))

,start_timestamp,end_timestamp,users,ads,campaigns,total_events
0,2025-05-07 14:11:57,2025-08-06 14:11:30,9950,200,48,400000


**Answer**  
The dataset covers from May 07, 2025 to August 06, 2025 and contains 400,000 events across 9,950 users, 200 ads, and 48 campaigns.

### Question
What is the total volume of events by type?

In [6]:
sql = r'''SELECT ad_events.event_type, COUNT(*) AS total_events
FROM ad_events
GROUP BY ad_events.event_type
ORDER BY total_events DESC;'''
display(q(sql))

,event_type,total_events
0,Impression,339812
1,Click,40079
2,Like,12013
3,Comment,4108
4,Purchase,2031
5,Share,1957


**Answer**  
Event volume by type is: Impression 339812, Click 40079, Like 12013, Comment 4108, Purchase 2031, Share 1957.

### Question
What is the overall conversion rate using purchases divided by clicks?

In [7]:
sql = r'''SELECT
  COUNT(*) FILTER (WHERE ad_events.event_type = 'Purchase')::DOUBLE
  / NULLIF(COUNT(*) FILTER (WHERE ad_events.event_type = 'Click'), 0) AS cvr
FROM ad_events;'''
display(q(sql))

,cvr
0,0.050675


**Answer**  
The overall conversion rate is 0.0507.

## Country Performance

### Question
Which country generated the highest number of purchases?

In [8]:
sql = r'''SELECT
  users.country AS country,
  COUNT(*) AS purchases
FROM ad_events
JOIN users
  ON ad_events.user_id = users.user_id
WHERE ad_events.event_type = 'Purchase'
  AND users.country IS NOT NULL
GROUP BY users.country
ORDER BY purchases DESC
LIMIT 20;'''
display(q(sql))

,country,purchases
0,United States,656
1,United Kingdom,290
2,Canada,201
3,India,189
4,Germany,152
5,Mexico,127
6,Japan,127
7,Australia,127
8,Brazil,119
9,France,62


**Answer**  
The top country by purchases is United States with 656 purchases.

### Question
Which are the three countries with the highest conversion rate?

In [9]:
sql = r'''SELECT
  users.country AS country,
  CAST(COUNT(*) FILTER (WHERE ad_events.event_type = 'Purchase') AS DOUBLE)
    / NULLIF(COUNT(*) FILTER (WHERE ad_events.event_type = 'Click'), 0) AS cvr
FROM ad_events
JOIN users ON ad_events.user_id = users.user_id
WHERE users.country IS NOT NULL
GROUP BY 1
HAVING COUNT(*) FILTER (WHERE ad_events.event_type = 'Click') > 0
ORDER BY cvr DESC
LIMIT 3;'''
display(q(sql))

,country,cvr
0,Japan,0.064763
1,Mexico,0.061680
2,United States,0.053368


**Answer**  
The three highest conversion rates are: Japan conversion rate of 0.0648, Mexico conversion rate of 0.0618, United States conversion rate of 0.0534.

### Question
Which countries have high click volume but low conversion efficiency?

In [10]:
sql = r'''SELECT
  users.country AS country,
  CAST(COUNT(*) FILTER (WHERE ad_events.event_type = 'Purchase') AS DOUBLE)
    / NULLIF(COUNT(*) FILTER (WHERE ad_events.event_type = 'Click'), 0) AS cvr
FROM ad_events
JOIN users ON ad_events.user_id = users.user_id
WHERE users.country IS NOT NULL
GROUP BY 1
HAVING COUNT(*) FILTER (WHERE ad_events.event_type = 'Click') > 0
ORDER BY COUNT(*) FILTER (WHERE ad_events.event_type = 'Click') DESC, cvr ASC
LIMIT 3;'''
display(q(sql))

,country,cvr
0,United States,0.053368
1,United Kingdom,0.048117
2,Canada,0.049192


**Answer**  
Among high click volume countries, the lowest conversion rates in this ranking are: United States conversion rate of 0.0534, United Kingdom conversion rate of 0.0481, Canada conversion rate of 0.0492.

## Campaign Performance

### Question
Which three campaigns generated the highest number of purchases?

In [11]:
sql = r'''SELECT
  campaigns.name AS campaign_name,
  COUNT(*) AS purchases
FROM ad_events
JOIN ads ON ad_events.ad_id = ads.ad_id
JOIN campaigns ON ads.campaign_id = campaigns.campaign_id
WHERE ad_events.event_type = 'Purchase'
GROUP BY 1
ORDER BY purchases DESC
LIMIT 3;'''
display(q(sql))

,campaign_name,purchases
0,Campaign_38_Q3,96
1,Campaign_17_Launch,91
2,Campaign_33_Summer,85


**Answer**  
The three campaigns with the most purchases are: Campaign38Q3 with 96 purchases, Campaign17Launch with 91 purchases, Campaign33Summer with 85 purchases.

### Question
Which three campaigns have the highest conversion rate?

In [12]:
sql = r'''SELECT
  campaigns.name AS campaign_name,
  CAST(COUNT(*) FILTER (WHERE ad_events.event_type = 'Purchase') AS DOUBLE)
    / NULLIF(COUNT(*) FILTER (WHERE ad_events.event_type = 'Click'), 0) AS cvr
FROM ad_events
JOIN ads ON ad_events.ad_id = ads.ad_id
JOIN campaigns ON ads.campaign_id = campaigns.campaign_id
GROUP BY 1
HAVING COUNT(*) FILTER (WHERE ad_events.event_type = 'Click') > 0
ORDER BY cvr DESC
LIMIT 3;'''
display(q(sql))

,campaign_name,cvr
0,Campaign_27_Q3,0.083969
1,Campaign_31_Summer,0.071307
2,Campaign_23_Winter,0.069421


**Answer**  
The three highest campaign conversion rates are: Campaign27Q3 with 0.084 conversion rate, Campaign31Summer conversion rate of 0.0713, Campaign23Winter conversion rate of 0.0694.

## Reusable view: events context

At this point, the same joins across events, ads, campaigns, and users begin to repeat. A reusable event level view is created to centralize campaign and country fields that will be used frequently in the next questions.


In [13]:
sql = r'''CREATE OR REPLACE VIEW events_context AS
SELECT
  ad_events.timestamp AS timestamp,
  DATE(ad_events.timestamp) AS event_date,
  ad_events.event_type AS event_type,
  ad_events.ad_id AS ad_id,
  ad_events.user_id AS user_id,
  ads.campaign_id AS campaign_id,
  campaigns.name AS campaign_name,
  users.country AS country
FROM ad_events
JOIN ads
  ON ad_events.ad_id = ads.ad_id
JOIN campaigns
  ON ads.campaign_id = campaigns.campaign_id
JOIN users
  ON ad_events.user_id = users.user_id;'''
con.execute(sql)

## Campaign and Country Consistency

### Question
Do top campaigns perform consistently across countries?

In [14]:
sql = r'''WITH campaign_totals AS (
  SELECT
    campaigns.campaign_id AS campaign_id,
    campaigns.name AS campaign_name,
    CAST(COUNT(*) FILTER (WHERE ad_events.event_type = 'Purchase') AS DOUBLE)
      / NULLIF(COUNT(*) FILTER (WHERE ad_events.event_type = 'Click'), 0) AS cvr
  FROM ad_events
  JOIN ads ON ad_events.ad_id = ads.ad_id
  JOIN campaigns ON ads.campaign_id = campaigns.campaign_id
  GROUP BY 1, 2
  HAVING COUNT(*) FILTER (WHERE ad_events.event_type = 'Click') > 0
),
top_campaigns AS (
  SELECT campaign_id, campaign_name
  FROM campaign_totals
  ORDER BY cvr DESC
  LIMIT 3
)
SELECT
  tc.campaign_name AS campaign_name,
  users.country AS country,
  CAST(COUNT(*) FILTER (WHERE ad_events.event_type = 'Purchase') AS DOUBLE)
    / NULLIF(COUNT(*) FILTER (WHERE ad_events.event_type = 'Click'), 0) AS cvr
FROM ad_events
JOIN ads ON ad_events.ad_id = ads.ad_id
JOIN top_campaigns tc ON ads.campaign_id = tc.campaign_id
JOIN users ON ad_events.user_id = users.user_id
WHERE users.country IS NOT NULL
GROUP BY 1, 2
HAVING COUNT(*) FILTER (WHERE ad_events.event_type = 'Click') > 0
ORDER BY campaign_name, cvr DESC;'''
display(q(sql))

,campaign_name,country,cvr
0,Campaign_23_Winter,France,0.130435
1,Campaign_23_Winter,India,0.096774
2,Campaign_23_Winter,Brazil,0.083333
3,Campaign_23_Winter,United States,0.078431
4,Campaign_23_Winter,Japan,0.071429
5,Campaign_23_Winter,United Kingdom,0.059524
6,Campaign_23_Winter,Mexico,0.050000
7,Campaign_23_Winter,Germany,0.050000
8,Campaign_23_Winter,Canada,0.050000
9,Campaign_23_Winter,Australia,0.044444


**Answer**  
Campaign27Q3 show noticeable variation in conversion performance across countries.

### Question
Which campaigns show the highest conversion rate variation across countries?

In [15]:
sql = r'''WITH campaign_country_cvr AS (
  SELECT
    events_context.campaign_id AS campaign_id,
    events_context.campaign_name AS campaign_name,
    events_context.country AS country,
    COUNT(*) FILTER (WHERE events_context.event_type = 'Purchase')::DOUBLE
      / NULLIF(COUNT(*) FILTER (WHERE events_context.event_type = 'Click'), 0) AS cvr
  FROM events_context
  WHERE events_context.country IS NOT NULL
  GROUP BY events_context.campaign_id, events_context.campaign_name, events_context.country
  HAVING COUNT(*) FILTER (WHERE events_context.event_type = 'Click') > 0
)
SELECT
  campaign_country_cvr.campaign_id AS campaign_id,
  campaign_country_cvr.campaign_name AS campaign_name,
  MAX(campaign_country_cvr.cvr) - MIN(campaign_country_cvr.cvr) AS cvr_range
FROM campaign_country_cvr
GROUP BY campaign_country_cvr.campaign_id, campaign_country_cvr.campaign_name
ORDER BY cvr_range DESC
LIMIT 20;'''
display(q(sql))

,campaign_id,campaign_name,cvr_range
0,49,Campaign_49_Winter,0.272727
1,35,Campaign_35_Launch,0.200000
2,41,Campaign_41_Winter,0.200000
3,6,Campaign_6_Winter,0.198511
4,45,Campaign_45_Summer,0.173913
5,1,Campaign_1_Launch,0.172414
6,31,Campaign_31_Summer,0.141667
7,2,Campaign_2_Launch,0.136364
8,21,Campaign_21_Winter,0.135135
9,14,Campaign_14_Summer,0.133333


 The campaign with the largest conversion rate variation across countries is Campaign49Winter at conversion rate of 0.2727. The variation across countries refers to differences in conversion performance for the same campaign in different markets.

## Cost and Efficiency Analysis

### Question
For the top three campaigns by conversion rate and the bottom three campaigns by conversion rate, what is the cost per acquisition?

In [16]:
sql = r'''WITH campaign_stats AS (
  SELECT
    campaigns.campaign_id AS campaign_id,
    campaigns.name AS campaign_name,
    campaigns.total_budget AS total_budget,
    COUNT(*) FILTER (WHERE ad_events.event_type = 'Click') AS clicks,
    COUNT(*) FILTER (WHERE ad_events.event_type = 'Purchase') AS purchases,
    CAST(COUNT(*) FILTER (WHERE ad_events.event_type = 'Purchase') AS DOUBLE)
      / NULLIF(COUNT(*) FILTER (WHERE ad_events.event_type = 'Click'), 0) AS cvr
  FROM ad_events
  JOIN ads ON ad_events.ad_id = ads.ad_id
  JOIN campaigns ON ads.campaign_id = campaigns.campaign_id
  GROUP BY 1, 2, 3
  HAVING COUNT(*) FILTER (WHERE ad_events.event_type = 'Click') > 0
),
top_cvr AS (
  SELECT campaign_id, campaign_name,
    total_budget / NULLIF(purchases, 0) AS cpa
  FROM campaign_stats
  ORDER BY cvr DESC
  LIMIT 3
),
bottom_cvr AS (
  SELECT campaign_id, campaign_name,
    total_budget / NULLIF(purchases, 0) AS cpa
  FROM campaign_stats
  ORDER BY cvr ASC
  LIMIT 3
)
SELECT 'Top 3 by conversion rate' AS segment, campaign_name, cpa
FROM top_cvr
UNION ALL
SELECT 'Bottom 3 by conversion rate' AS segment, campaign_name, cpa
FROM bottom_cvr
ORDER BY segment, cpa;'''
display(q(sql))

,segment,campaign_name,cpa
0,Bottom 3 by conversion rate,Campaign_19_Winter,2370.175000
1,Bottom 3 by conversion rate,Campaign_32_Summer,3027.575185
2,Bottom 3 by conversion rate,Campaign_36_Q3,5880.199000
3,Top 3 by conversion rate,Campaign_27_Q3,393.524242
4,Top 3 by conversion rate,Campaign_23_Winter,619.087381
5,Top 3 by conversion rate,Campaign_31_Summer,1719.246190


**Answer**  
Top three by conversion rate cost per acquisition values are: Campaign27Q3 with 393.52, Campaign23Winter cost per acquisition of 619.09, Campaign31Summer cost per acquisition of 1719.25. Bottom three by conversion rate cost per acquisition values are: Campaign36Q3 with 5880.20, Campaign32Summer cost per acquisition of 3027.58, Campaign19Winter cost per acquisition of 2370.17.

## Reusable view: campaign metrics

From this point forward, several questions repeat campaign level aggregations such as volumes, rates, and cost efficiency. A reusable campaign level view is created to centralize the main metrics per campaign.


In [17]:
sql = r'''CREATE OR REPLACE VIEW campaign_metrics AS
WITH campaign_events AS (
  SELECT
    events_context.campaign_id AS campaign_id,
    events_context.campaign_name AS campaign_name,
    COUNT(*) FILTER (WHERE events_context.event_type = 'Impression') AS impressions,
    COUNT(*) FILTER (WHERE events_context.event_type = 'Click') AS clicks,
    COUNT(*) FILTER (WHERE events_context.event_type = 'Purchase') AS purchases
  FROM events_context
  GROUP BY events_context.campaign_id, events_context.campaign_name
)
SELECT
  campaigns.campaign_id AS campaign_id,
  campaigns.name AS campaign_name,
  campaigns.start_date AS start_date,
  campaigns.end_date AS end_date,
  campaigns.total_budget AS total_budget,
  campaign_events.impressions AS impressions,
  campaign_events.clicks AS clicks,
  campaign_events.purchases AS purchases,
  campaign_events.clicks::DOUBLE / NULLIF(campaign_events.impressions, 0) AS ctr,
  campaign_events.purchases::DOUBLE / NULLIF(campaign_events.clicks, 0) AS cvr,
  campaigns.total_budget::DOUBLE / NULLIF(campaign_events.purchases, 0) AS cpa
FROM campaigns
JOIN campaign_events
  ON campaigns.campaign_id = campaign_events.campaign_id;'''
con.execute(sql)

## Diagnostics and Prioritization

### Question
Which campaigns have high budget and low purchases?

In [18]:
sql = r'''SELECT
  campaign_metrics.campaign_id AS campaign_id,
  campaign_metrics.campaign_name AS campaign_name,
  campaign_metrics.total_budget AS total_budget,
  campaign_metrics.purchases AS purchases,
  campaign_metrics.cpa AS cpa
FROM campaign_metrics
ORDER BY total_budget DESC, purchases ASC
LIMIT 20;'''
display(q(sql))

,campaign_id,campaign_name,total_budget,purchases,cpa
0,20,Campaign_20_Winter,98904.66,73,1354.858356
1,46,Campaign_46_Winter,94023.76,36,2611.771111
2,17,Campaign_17_Launch,86675.92,91,952.482637
3,15,Campaign_15_Launch,85407.23,42,2033.505476
4,41,Campaign_41_Winter,85220.35,36,2367.231944
5,32,Campaign_32_Summer,81744.53,27,3027.575185
6,2,Campaign_2_Launch,79342.41,31,2559.432581
7,11,Campaign_11_Q3,79067.91,37,2136.970541
8,6,Campaign_6_Winter,78607.49,33,2382.045152
9,30,Campaign_30_Winter,73911.29,42,1759.792619


**Answer**  
The highest budget campaigns that still show low purchase volume in this ranking are: Campaign20Winter (budget of 98904.66, purchases 73), Campaign46Winter (budget of 94023.76, purchases 36), Campaign17Launch (budget of 86675.92, purchases 91).

### Question
Which campaigns combine strong purchase volume with low cost per acquisition?

In [19]:
sql = r'''SELECT
  campaign_metrics.campaign_id AS campaign_id,
  campaign_metrics.campaign_name AS campaign_name,
  campaign_metrics.purchases AS purchases,
  campaign_metrics.total_budget AS total_budget,
  campaign_metrics.cpa AS cpa,
  campaign_metrics.cvr AS cvr
FROM campaign_metrics
ORDER BY purchases DESC, cpa ASC
LIMIT 20;'''
display(q(sql))

,campaign_id,campaign_name,purchases,total_budget,cpa,cvr
0,38,Campaign_38_Q3,97,71038.28,732.353402,0.060248
1,17,Campaign_17_Launch,91,86675.92,952.482637,0.054556
2,33,Campaign_33_Summer,86,59264.68,689.124186,0.060777
3,24,Campaign_24_Summer,81,56692.87,699.911975,0.050943
4,20,Campaign_20_Winter,73,98904.66,1354.858356,0.043660
5,34,Campaign_34_Winter,69,26104.30,378.323188,0.055156
6,42,Campaign_42_Summer,67,7918.04,118.179701,0.041333
7,13,Campaign_13_Winter,65,21855.42,336.237231,0.053018
8,9,Campaign_9_Launch,63,40094.07,636.413810,0.050930
9,47,Campaign_47_Launch,63,69493.36,1103.069206,0.052023


**Answer**  
Campaigns combining high purchase volume with low cost per acquisition include: Campaign38Q3 (purchases 97, cost per acquisition cost per acquisition of 732.35), Campaign1Launch (purchases 91, cost per acquisition cost per acquisition of 952.48), Campaign33Summer (purchases 86, cost per acquisition cost per acquisition of 689.12).

### Question
Do any countries concentrate a high number of low performing campaigns?

In [20]:
sql = r'''WITH campaign_country AS (
  SELECT
    users.country AS country,
    campaigns.campaign_id AS campaign_id,
    campaigns.name AS campaign_name,
    COUNT(*) FILTER (WHERE ad_events.event_type = 'Click') AS clicks,
    COUNT(*) FILTER (WHERE ad_events.event_type = 'Purchase') AS purchases,
    CAST(COUNT(*) FILTER (WHERE ad_events.event_type = 'Purchase') AS DOUBLE)
      / NULLIF(COUNT(*) FILTER (WHERE ad_events.event_type = 'Click'), 0) AS cvr
  FROM ad_events
  JOIN users ON ad_events.user_id = users.user_id
  JOIN ads ON ad_events.ad_id = ads.ad_id
  JOIN campaigns ON ads.campaign_id = campaigns.campaign_id
  WHERE users.country IS NOT NULL
  GROUP BY 1, 2, 3
  HAVING COUNT(*) FILTER (WHERE ad_events.event_type = 'Click') >= 20
),
threshold AS (
  SELECT PERCENTILE_CONT(0.25) WITHIN GROUP (ORDER BY cvr) AS low_cvr_threshold
  FROM campaign_country
)
SELECT
  cc.country AS country,
  COUNT(DISTINCT cc.campaign_id) AS low_performing_campaigns
FROM campaign_country cc
CROSS JOIN threshold t
WHERE cc.cvr <= t.low_cvr_threshold
GROUP BY 1
ORDER BY low_performing_campaigns DESC
LIMIT 5;'''
display(q(sql))

,country,low_performing_campaigns
0,Brazil,16
1,France,14
2,Australia,14
3,Mexico,13
4,Germany,12


**Answer**  
Yes. The highest concentrations of low performing campaigns are in: Brazil 16, Australia 14, France 14.

### Dashboard Overview

In addition to the analytical questions explored in this project, an interactive dashboard was developed to support a high level view of marketing performance. The dashboard focuses on campaign results across countries and platforms, highlighting how impressions and purchases are distributed across different segments.

This complementary view allows for a more visual exploration of performance patterns, making it easier to compare campaigns and markets at a glance and identify areas of concentration or imbalance. The dashboard is intended to support quick assessment and communication of results rather than replace the detailed question driven analysis presented above.

https://public.tableau.com/views/MarketingPerformanceAnalysis_17678625116620/MarketingPerformance?:language=pt-BR&:sid=&:redirect=auth&:display_count=n&:origin=viz_share_link


## Conclusion

The analysis highlights that campaign performance and efficiency are not interchangeable concepts. Campaigns that generate a high number of purchases are not necessarily the most efficient in terms of conversion rate or cost per acquisition, reinforcing the importance of evaluating volume based metrics alongside efficiency metrics when assessing overall effectiveness.

Additionally, aggregated conversion results can obscure meaningful local differences. While some campaigns perform well at a global level, their effectiveness varies significantly across countries, indicating that strong overall performance may coexist with underperformance in specific markets. This variation emphasizes the need to look beyond aggregated metrics to fully understand campaign behavior.

The findings also underline the importance of geographic segmentation in marketing analysis. Differences in user behavior across countries play a substantial role in campaign outcomes, suggesting that market specific factors influence conversion performance and should be considered when evaluating results.


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5562491b-a98e-4925-a811-76400f1e61c7' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>